In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/4.vehicle'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/3.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input = df_input[df_input['Vehicle Url'] != ''].reset_index(drop=True)

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        (crawler_index, crawler_series) = crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            list_dict_correct_temp, page = [], 0
            while True:

                count_retry = 0
                while True:
                    count_retry += 1

                    try:
                        url_request = f'''{crawler_series['Vehicle Url']}&num=50&start={page * 50}'''

                        resp = requests.get(url_request,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(5, 15))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            if html.xpath('//div[@class="detail-app-pager"]'):
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                next_ = html.xpath('//a[@id="pagingBottom_nextButton"]')
                if page == 0 and next_ and next_[0].xpath('./@href')[0].strip() == 'Search/#':
                    raise

                # = = = = = = = = = = = = = = =

                list_text, list_tr = [text.strip() for text in html.xpath('//th[@scope="col"]/text()')], html.xpath('//tr[@class="detail-app-row"]')

                # = = = = = = = = = = = = = = =

                list_dict_correct_temp_temp = [{'Vehicle Url': crawler_series['Vehicle Url'],
                                                'Page': page + 1,
                                                'Row': i + 1} for i in range(len(list_tr))]

                for i, tr in enumerate(list_tr):
                    for j, text in enumerate(list_text):
                        list_dict_correct_temp_temp[i][text] = tr.xpath(f'./td[{j+1}]/text()')[0].strip()

                list_dict_correct_temp.extend(list_dict_correct_temp_temp)

                # = = = = = = = = = = = = = = =

                if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 0 or count_remain % 30 == 0 and count_remain != 0:
                    clear_output()
                print(f'''[第{page + 1}页] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

                # = = = = = = = = = = = = = = =

                if not next_:
                    break

                # = = = = = = = = = = = = = = =

                page += 1

            # = = = = = = = = = = = = = = =

            if not list_dict_correct_temp:
                raise

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.extend(list_dict_correct_temp)
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Vehicle Url', 'Page', 'Row'],
                                                                 ascending=[True, True, True],
                                                                 ignore_index=True)
        df_correct['Year'] = df_correct['Year'].astype(int)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)}''')

print('Done ~')

总数量：10

[第1页] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/15 19:26:36] - 3. https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=49997&PartType=Brake%20Backing%20Plate&num=50&start=0

[第1页] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/15 19:26:36] - 2. https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=49987&PartType=Brake%20Backing%20Plate&num=50&start=0

[第1页] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/15 19:26:36] - 5. https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=137413&PartType=Brake%20Backing%20Plate&num=50&start=0

[第2页] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/15 19:26:38] - 2. https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=49987&PartType=Brake%20Backing%20Plate&num=50&start=50

[第2页] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/15 19:26:38] - 5. https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=137413&PartType=Brake%20Backing%20Plate&num=50&start=50

[第1页] - [请求2次] - [剩余8条] - [crawler_1（5

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4443.12it/s]

Done ~
